In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv("./new_preprocessed_final.csv")

In [3]:
# TF‑IDF / token overlap
tfidf_feats = [c for c in df.columns if ("cosine" in c.lower() and "bert" not in c.lower()) or "overlap" in c.lower()]

# BERT similarity
bert_feat = None
for c in df.columns:
  if "bert_similarity" in c.lower() or ("cosine" in c.lower() and "bert" in c.lower()):
    bert_feat = c
    break

# Extra features
extra_feats = ["title_length", "sensational_word_count", "punctuation_count", "punctuation_ratio"]
extra_feats = [c for c in extra_feats if c in df.columns]

# Combine all
feature_cols = tfidf_feats + ([bert_feat] if bert_feat else []) + extra_feats

# Label
label_col = None
for c in df.columns:
  if "is_clickbait" in c.lower():
    label_col = c
    break

X = df[feature_cols]
y = df[label_col]

# Train‑test split
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=42, stratify=y
)

# Random Forest
rf = RandomForestClassifier(
  n_estimators=300,
  max_depth=None,
  class_weight="balanced",
  random_state=42
)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("=== Features Used ===")
print(feature_cols)

print("\n=== Classification Report (RF + TF‑IDF + BERT + Extras) ===")
print(classification_report(y_test, y_pred))

print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

=== Features Used ===
['cosine_similarity', 'token_overlap', 'bert_similarity', 'title_length', 'sensational_word_count', 'punctuation_count', 'punctuation_ratio']

=== Classification Report (RF + TF‑IDF + BERT + Extras) ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       514
           1       1.00      1.00      1.00        86

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600


=== Confusion Matrix ===
[[514   0]
 [  0  86]]
